# Weekly Programming Exercise B1 - EX 9

Hooray!  It's time for a wedding!

Fine, so it's only a virtual wedding. But we have a bunch of guests, and we need to organize them in some way.  We're going to do that with a GuestList class. We'll create an instance of this class, and add our guests (as named tuples) into the guest list, along with a table number. We'll then be able to ask our class for a complete guest list by table, as well as a report of which tables aren't yet full.

Here's how the class should work:
    # Person should be a named tuple, with "first" and "last" attributes
    
    gl = GuestList()

    gl.assign(Person('Waylon', 'Dalton'), 1)
    gl.assign(Person('Justine', 'Henderson'), 1)
    gl.assign(Person('Abdullah', 'Lang'), 3)
    gl.assign(Person('Marcus', 'Cruz'), 1)
    gl.assign(Person('Thalia', 'Cobb'), 2)
    gl.assign(Person('Mathias', 'Little'), 2)
    gl.assign(Person('Eddie', 'Randolph'), None)
    gl.assign(Person('Angela', 'Walker'), 2)
    gl.assign(Person('Lia', 'Shelton'), 3)
    gl.assign(Person('Hadassah', 'Hartman'), None)
    gl.assign(Person('Joanna', 'Shaffer'), 3)
    gl.assign(Person('Jonathon', 'Sheppard'), 2)

In the above code, I've created a guest list.  Each Person is then created and assigned to a table.  If we assign a person to table None, they aren't assigned.

We will assume, for the purposes of this exercise, that everyone in the world has a unique name.

We will also assume that a maximum of 10 people can be added to a given table.  Attempting to add more than 10 people to a table results in a TableFull exception.

Now we want to run a few reports on our guest list:

(1) How many guests are there, total?  We should be able to run
    len(gl)

to get an integer back.

(2) What guests are at a given table?  We should be able to run
    gl.table(2)

and get a list of Person objects back.

(3) What guests aren't assigned to any table?  We should be able to run
    gl.unassigned()

and get a list of Person objects back.

(4) Given a Person object, we should be able to assign them to a table:
    p = Person('Joanna', 'Shaffer')
    gl.assign(p, 3)

If the person is already in the system, but is assigned to another table (or to no table at all), then they will now be assigned to a new table.

If the person isn't already in the system, then they will be added and then assigned to a table.

If there is no room at the table (i.e., there are already 10 guests there), then we should raise a TableFull exception.

(5) We should also be able to learn how much space is avaialble at each table:
    gl.free_space()

The above should return a dictionary of table names (keys) and remaining space (values) for each table.

(6) We should be able to say
    gl.guests()

and get a list of all guests, sorted first by table number, then by last name, and finally by first name.

(7) Finally, we should be able to say
    print(gl)

and get the above list printed nicely.  I'm thinking something like:
    
    2
        last, first
        last, first
    
    3
        last, first
        last, first
    
    4
        last, first
        last, first
        last, first
    
Who said weddings are easy?

I'll be back on Monday with my suggested solutions.  Meanwhile, you can discuss it in the forum!

Reuven

In [1]:
from collections import namedtuple
Person = namedtuple('Person', 'first last')

In [2]:
# let's define our custom exception
class TableFull(Exception):
    '''Raised when attempting to add a Person to a table that alread has 10 seated'''
    pass

In [3]:
class GuestList():
    max_at_table = 10               #<-- establish our maximum per table
    
    def __init__(self):
        self.allguests = list()              #<-- establish our guestlist
           
    def assign(self, person, table):         #<-- method for adding/assigning, let's try and manage this only with list of lists
        
        entry = [person, table]              #<-- create entry
        exists = False                       #<-- initialize exists as False until we lookup
        
        for idx, guests in enumerate(self.allguests): #<-- check if person exists, and get index of where they exist
            if person == guests[0]:
                exists = True
                break
        
        ## If the person DOES exist and they have a table number in the new assignment
        existing_table_count = 0             #<-- init existing_table_count to check current table occupancy
        if exists == True and table != None:     
            for guest in self.allguests:     #<-- check destination table occupancy
                if guest[1] == table:
                    existing_table_count += 1
            if existing_table_count < self.max_at_table:    #<-- if there's room, make update
                self.allguests.pop(idx)         #<-- remove previous entry for guest
                self.allguests.append(entry)    #<-- add new entry to guestlist
            else:                            #<-- otherwise, no room, throw exception
                raise TableFull
        
        ## If the person DOES NOT exist and they have a table number in the new assignment
        existing_table_count = 0             #<-- init existing_table_count to check current table occupancy
        if exists == False and table != None: #<-- get existing table count for entry table
            for guest in self.allguests:    #<-- check destination table occupancy
                if guest[1] == table:
                    existing_table_count += 1
            if existing_table_count < self.max_at_table:       #<-- if there's room, make update
                self.allguests.append(entry)    #<-- add to guestlist
            else:
                raise TableFull               #<-- otherwise, no room, throw exception
        
        ## If the person DOES exist and they have None in the new table assignment
        if exists == True and table == None:
            self.allguests.pop(idx)         #<-- remove previous entry for guest
            self.allguests.append(entry)    #<-- add new entry to guestlist
        
        ## If the person DOES NOT exist and they have None in the new table assignment
        if exists == False and table == None:
            self.allguests.append(entry)    #<-- add to guestlist no validation as no table assignment 
    
    def unassigned(self):                    #<-- returns list of people with None assigned as table
        return [each[0] for each in self.allguests if each[1] is None]  
   
    def table(self, table):                  #<-- returns list of people assigned to table
        return [each[0] for each in self.allguests if each[1] == table] 
    
    def free_space(self):                    #<-- returns free space for each table
        remaining_space = dict()
        for guest in self.allguests:
            if guest[1] in remaining_space.keys():
                remaining_space[guest[1]] = remaining_space[guest[1]] - 1 #<-- table exists, so substract a seat
            else: 
                remaining_space[guest[1]] = self.max_at_table - 1 #<-- table doesn't exist in dict(), set to self.max_at_table minus 1
        remaining_space.pop(None, None)           #<-- let's remove the None key here, doesn't make sense
        return remaining_space
    
    def __len__(self):                       #<-- implement our own len for our objects
        return len(self.allguests)

    def guests(self):
        assignedguests = [guest for guest in self.allguests if guest[1] is not None]   #<-- let's pop the None's off...for now. we might add them back to this list
        sortedguests = sorted(assignedguests, key = lambda x: (x[1], x[0].last, x[0].first))
        return [guest[0] for guest in sortedguests]    # return only the Person tuple, not the table per requirements
    
    
    def __str__(self):
        
        # borrowed this from guests(), because i need a version with both Person and table number
        assignedguests = [guest for guest in self.allguests if guest[1] is not None]   #<-- let's pop the None's off...for now. we might add them back to this list
        guestssorted = sorted(assignedguests, key = lambda x: (x[1], x[0].last, x[0].first))
        
        printout = ''
        
        currenttable = guestssorted[0][1]        #<-- get the first table 
        printout += str(currenttable) + '\n'
        
        for guest in guestssorted:
            if guest[1] != currenttable:
                printout += str(guest[1]) + '\n'
            currenttable = guest[1]
            printout += f'\t{guest[0].last}, {guest[0].first}\n'               
        return printout

In [4]:
gl = GuestList()
gl.assign(Person('Waylon', 'Dalton'), 1)
gl.assign(Person('Justine', 'Henderson'), 1)
gl.assign(Person('Abdullah', 'Lang'), 3)
gl.assign(Person('Marcus', 'Cruz'), 1)
gl.assign(Person('Thalia', 'Cobb'), 2)
gl.assign(Person('Mathias', 'Little'), 2)
gl.assign(Person('Eddie', 'Randolph'), 4)
gl.assign(Person('Angela', 'Walker'), 2)
gl.assign(Person('Lia', 'Shelton'), 3)
gl.assign(Person('Hadassah', 'Hartman'), 4)
gl.assign(Person('Joanna', 'Shaffer'), 3)
gl.assign(Person('Benjamin', 'Shaffer'), 3)
gl.assign(Person('William', 'Shaffer'), 3)
gl.assign(Person('Jonathon', 'Sheppard'), 2)
gl.assign(Person('dfsgfg', 'wer'), 2)
gl.assign(Person('Eddjkgie', 'dfssg'), None)
gl.assign(Person('Anasdfdsfgela', 'bnm'), None)

In [5]:
gl.max_at_table

10

In [6]:
print(gl)

1
	Cruz, Marcus
	Dalton, Waylon
	Henderson, Justine
2
	Cobb, Thalia
	Little, Mathias
	Sheppard, Jonathon
	Walker, Angela
	wer, dfsgfg
3
	Lang, Abdullah
	Shaffer, Benjamin
	Shaffer, Joanna
	Shaffer, William
	Shelton, Lia
4
	Hartman, Hadassah
	Randolph, Eddie



In [7]:
gl.assign(Person('Waylon', 'Dalton'), 3)

In [8]:
gl.guests()

[Person(first='Marcus', last='Cruz'),
 Person(first='Justine', last='Henderson'),
 Person(first='Thalia', last='Cobb'),
 Person(first='Mathias', last='Little'),
 Person(first='Jonathon', last='Sheppard'),
 Person(first='Angela', last='Walker'),
 Person(first='dfsgfg', last='wer'),
 Person(first='Waylon', last='Dalton'),
 Person(first='Abdullah', last='Lang'),
 Person(first='Benjamin', last='Shaffer'),
 Person(first='Joanna', last='Shaffer'),
 Person(first='William', last='Shaffer'),
 Person(first='Lia', last='Shelton'),
 Person(first='Hadassah', last='Hartman'),
 Person(first='Eddie', last='Randolph')]

In [9]:
gl.unassigned()

[Person(first='Eddjkgie', last='dfssg'),
 Person(first='Anasdfdsfgela', last='bnm')]

In [10]:
gl.table(3)

[Person(first='Abdullah', last='Lang'),
 Person(first='Lia', last='Shelton'),
 Person(first='Joanna', last='Shaffer'),
 Person(first='Benjamin', last='Shaffer'),
 Person(first='William', last='Shaffer'),
 Person(first='Waylon', last='Dalton')]

In [11]:
gl.table(2)

[Person(first='Thalia', last='Cobb'),
 Person(first='Mathias', last='Little'),
 Person(first='Angela', last='Walker'),
 Person(first='Jonathon', last='Sheppard'),
 Person(first='dfsgfg', last='wer')]

In [12]:
gl.free_space()

{1: 8, 3: 4, 2: 5, 4: 8}

In [13]:
gl.assign(Person('sdfd', 'Daltervon'), 1)
gl.assign(Person('Waadsfylon', 'Dadflton'), 1)
gl.assign(Person('Waygfhgflon', 'Dalteon'), 1)
gl.assign(Person('Waygfhfglon', 'Dalsdfton'), 1)
gl.assign(Person('Waygfhjlon', 'Daltkjhon'), 1)
gl.assign(Person('Waysdaflon', 'Daltjhkon'), 1)
gl.assign(Person('new', 'person2'), 1)

In [14]:
gl.free_space()

{1: 1, 3: 4, 2: 5, 4: 8}

In [15]:
gl.assign(Person('Eddie', 'Randolph234'), 1)

In [16]:
gl.free_space()

{1: 0, 3: 4, 2: 5, 4: 8}

In [17]:
# throw error...table full
gl.assign(Person('Kim', 'Wilke'), 1)

TableFull: 